### Assignment 3

### Capital Bikeshare Data
https://ride.capitalbikeshare.com/system-data

In [115]:
# Reading the file directly from local
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

In [116]:
import pandas as pd
# Read csv files

df_Feb = pd.read_csv('202402-capitalbikeshare-tripdata.csv') # Updated path to read file from local
df_Mar = pd.read_csv('202403-capitalbikeshare-tripdata.csv') # Updated path to read file from local
df_Apr = pd.read_csv('202404-capitalbikeshare-tripdata.csv') # Updated path to read file from local

# concat data
df=pd.concat([df_Feb, df_Mar,df_Apr])

In [117]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,24DBACC298E33BF6,electric_bike,2024-02-01 21:18:50,2024-02-01 21:27:36,Eckington Pl & Q St NE,31505.0,5th & K St NW,31600.0,38.910893,-77.005023,38.90304,-77.019027,member
1,FE1067800A18BC56,electric_bike,2024-02-06 17:15:33,2024-02-06 17:30:33,4th & C St SW,31243.0,NaN,NaN,38.886443,-77.017388,38.91000,-76.980000,member
2,AD1962B215B5879D,electric_bike,2024-02-26 13:19:42,2024-02-26 13:30:44,North Capitol St & F St NW,31624.0,NaN,NaN,38.896907,-77.009684,38.90000,-77.010000,member
3,3F8352E6D9EC4F85,electric_bike,2024-02-02 11:01:22,2024-02-02 11:05:14,1st & K St NE,31662.0,NaN,NaN,38.902394,-77.005614,38.90000,-77.010000,member
4,D656DA87FA915F14,electric_bike,2024-02-25 08:28:18,2024-02-25 08:48:44,Virginia Ave & 25th St NW,31297.0,NaN,NaN,38.898456,-77.053075,38.91000,-77.060000,member


## Let's consider the GWSB station "22nd & H St NW"

### Bike Availability: Number of Pickups

In [118]:
# Convert the 'started_at' column to datetime objects
df['started_at_date'] = pd.to_datetime(df['started_at']).dt.date

# Filter for trips started at '22nd & H St NW'
PU_trips = df[df['start_station_name'] == '22nd & H St NW']

# Group by date and count the trips
PU_counts = PU_trips.groupby('started_at_date')['started_at_date'].count()

PU_counts

started_at_date
2024-02-01    33
2024-02-02    23
2024-02-03    22
2024-02-04    18
2024-02-05    19
              ..
2024-04-26    33
2024-04-27    18
2024-04-28    36
2024-04-29    37
2024-04-30    40
Name: started_at_date, Length: 90, dtype: int64

### Dock Availability: Number of Dropoffs

In [119]:
# Convert the 'ended_at' column to datetime objects
df['ended_at_date'] = pd.to_datetime(df['ended_at']).dt.date

# Filter for trips ended at '22nd & H St NW'
DO_trips = df[df['end_station_name'] == '22nd & H St NW']

# Group by date and count the trips
DO_counts = DO_trips.groupby('ended_at_date')['ended_at_date'].count()

DO_counts

ended_at_date
2024-02-01    33
2024-02-02    25
2024-02-03    20
2024-02-04    19
2024-02-05    23
              ..
2024-04-26    39
2024-04-27    24
2024-04-28    28
2024-04-29    38
2024-04-30    39
Name: ended_at_date, Length: 90, dtype: int64

## Merge PU and DO tables
- For each day, we look at the numbers of pickups and dropoffs

In [120]:
# merge PU_counts and DO_counts by matching the dates, and change the column names to PU_count and DO_count respectively.

PU_DO_counts = pd.merge(PU_counts, DO_counts, left_index=True, right_index=True, how='outer')
PU_DO_counts = PU_DO_counts.rename(columns={'started_at_date': 'PU_ct', 'ended_at_date': 'DO_ct'})

# change the index name from 'started_at' to 'date'

PU_DO_counts = PU_DO_counts.rename_axis('date')
PU_DO_counts

,PU_ct,DO_ct
date,,
2024-02-01,33,33
2024-02-02,23,25
2024-02-03,22,20
2024-02-04,18,19
2024-02-05,19,23
...,...,...
2024-04-26,33,39
2024-04-27,18,24
2024-04-28,36,28


# Feature Information: Weather Data
- From https://www.visualcrossing.com/weather-history/

In [121]:
df_weather = pd.read_csv('DC_weather_2024.csv')

df_weather.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"washington, dc",2024-01-01,43.0,38.9,40.8,41.8,31.4,36.4,33.3,74.7,...,0.9,1,10,2024-01-01T07:26:50,2024-01-01T16:56:30,0.68,Overcast,Cloudy skies throughout the day.,cloudy,"KDCA,72405013743,D6279,72403793728,KADW,KDAA,7..."
1,"washington, dc",2024-01-02,45.9,35.7,40.4,40.9,27.2,33.8,26.2,57.3,...,8.3,5,10,2024-01-02T07:26:58,2024-01-02T16:57:20,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KDCA,72405013743,D6279,72403793728,KADW,KDAA,7..."
2,"washington, dc",2024-01-03,47.8,33.7,39.9,44.6,27.8,36.4,26.6,59.5,...,9.6,5,10,2024-01-03T07:27:04,2024-01-03T16:58:10,0.75,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"C6294,KDCA,72405013743,D6279,72403793728,KADW,..."
3,"washington, dc",2024-01-04,46.7,33.2,40.4,41.9,24.1,34.6,25.2,55.2,...,4.8,4,10,2024-01-04T07:27:07,2024-01-04T16:59:02,0.78,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"C6294,KDCA,72405013743,72403793728,KADW,KDAA,7..."
4,"washington, dc",2024-01-05,39.9,27.8,33.6,35.8,19.4,28.0,18.9,55.0,...,11.6,5,10,2024-01-05T07:27:09,2024-01-05T16:59:55,0.81,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"C6294,KDCA,72405013743,72403793728,D6279,KADW,..."


## Drop variables of your choice

In [122]:
# Drop unnecessary variables
df_weather=df_weather.drop(columns=['name', 'stations','description','sunrise','sunset','conditions','severerisk','preciptype','windgust'])

## Prepare X and y: Merge df_weather with PU_DO_counts

In [123]:
# Convert the 'datetime' column to datetime objects and extract the date
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
df_weather['date'] = df_weather['datetime'].dt.date

# Merge the two dataframes based on the 'date' column
merged_df = pd.merge(PU_DO_counts, df_weather, on='date', how='left')

# Prepare training and test data

In [124]:
# we have two target variables: PU_ct and DO_ct
y = merged_df[['PU_ct','DO_ct']]
X_all = merged_df.drop(columns=['PU_ct','DO_ct']) # all features
X_all

,date,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase,icon
0,2024-02-01,2024-02-01,53.0,31.7,42.4,53.0,26.4,39.3,34.0,74.0,...,12.0,190.8,1016.3,43.3,8.4,136.8,11.8,5,0.73,partly-cloudy-day
1,2024-02-02,2024-02-02,51.8,44.0,47.1,51.8,36.6,44.4,35.5,65.1,...,19.1,354.3,1012.8,97.3,9.8,51.7,4.4,2,0.75,rain
2,2024-02-03,2024-02-03,51.0,35.8,42.2,51.0,28.0,37.6,26.2,53.8,...,14.7,354.9,1018.2,11.0,9.9,143.9,12.5,6,0.79,clear-day
3,2024-02-04,2024-02-04,51.2,32.7,41.7,51.2,27.4,40.7,26.0,56.4,...,9.2,87.4,1016.0,9.0,9.9,147.4,12.8,6,0.83,clear-day
4,2024-02-05,2024-02-05,51.8,32.5,41.0,51.8,26.0,37.8,26.0,57.4,...,12.2,34.9,1013.8,21.4,9.9,145.3,12.7,6,0.86,partly-cloudy-day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-04-26,2024-04-26,64.9,45.5,55.2,64.9,41.5,54.2,34.9,50.8,...,15.4,116.4,1029.5,69.1,9.9,215.0,18.5,9,0.59,partly-cloudy-day
86,2024-04-27,2024-04-27,60.1,51.9,55.7,60.1,51.9,55.7,45.4,69.0,...,10.0,149.9,1030.1,97.3,9.7,44.0,3.9,2,0.63,rain
87,2024-04-28,2024-04-28,83.1,55.8,67.0,83.3,55.8,67.1,56.4,70.6,...,10.0,170.0,1021.3,62.1,9.8,213.6,18.5,9,0.67,partly-cloudy-day
88,2024-04-29,2024-04-29,88.3,62.7,75.2,88.9,62.7,75.3,60.9,63.5,...,13.0,177.3,1015.6,35.8,9.9,204.5,17.8,9,0.70,partly-cloudy-day


## Reduce dimensions by feature type

In [125]:
#dimentionality reduction: PCA
def dim_red(dataframe,lists,columnname):
    #extract the columns
    datagroup=dataframe.filter(lists)
    #normalize the data
    from sklearn.preprocessing import StandardScaler
    data_norm = StandardScaler().fit_transform(datagroup)
    #feature extraction with PCA
    from sklearn.decomposition import PCA
    pca = PCA(n_components=1)
    PCs = pca.fit_transform(datagroup)

    df_pca = pd.DataFrame(data = PCs, columns = [columnname])
    return df_pca

In [126]:
temp_group=['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike','dew']
precip_group=['humidity','precip', 'precipprob', 'precipcover']
vis_group=['visibility','solarradiation', 'solarenergy', 'uvindex']
wind_group=['windgust','windspeed', 'winddir']

In [127]:
temp_PC1=dim_red(X_all,temp_group,'temp_PC1')
precip_PC1=dim_red(X_all,precip_group,'precip_PC1')
vis_PC1=dim_red(X_all,vis_group,'vis_PC1')
wind_PC1=dim_red(X_all,wind_group,'wind_PC1')

## Dimension reduction on the features
- Now we call the following features X

In [128]:
X = pd.concat([temp_PC1,precip_PC1,vis_PC1,wind_PC1],axis = 1)
X

,temp_PC1,precip_PC1,vis_PC1,wind_PC1
0,-23.616810,-32.623727,4.466466,-0.835152
1,-11.905681,62.494225,-80.995297,162.790919
2,-26.760771,-36.797274,11.641535,163.281210
3,-26.853470,-36.260085,15.152105,-104.272810
4,-28.487066,-36.053473,13.052631,-156.681869
...,...,...,...,...
85,7.209137,-37.417107,83.053732,-75.127468
86,14.738853,63.300059,-88.705521,-41.772262
87,47.863874,-33.326205,81.659100,-21.678490
88,65.008395,-34.793145,72.539192,-14.306077


## <font color="red"> Important: train_test_split with y=['PU_ct','DO_ct']. Why? <font>

In [129]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,y, test_size=0.4, random_state = 200) # 40% test data

In [130]:
y_train.head()

,PU_ct,DO_ct
73,26,24
60,26,22
18,10,15
44,19,20
4,19,23


# <font color="red"> Your code starts from here... </font>
- In this assignment, we first reduce the dimensions and then use only [temp_PC1,precip_PC1,vis_PC1,wind_PC1] for prediction.
- Follow the discussion and details in Lecture 6.
- Based on the analysis and result, summarize a 5-page final report in PDF.
- Submit the final report with Python file.

In [131]:
# Extra imports used only for the ML pipeline
import numpy as np
import pandas as pd
from itertools import product

from sklearn.preprocessing   import StandardScaler
from sklearn.pipeline        import Pipeline
from sklearn.metrics         import mean_squared_error
from sklearn.model_selection import KFold, RandomizedSearchCV

from sklearn.linear_model    import LinearRegression, Ridge, ElasticNet
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.multioutput     import MultiOutputRegressor

from scipy.stats import randint, uniform, loguniform

In [132]:
# Helper to compute business cost with α = 2 and β = 3 fixed
ALPHA, BETA = 2, 3          # penalty for missed pickup / drop-off

def decision_cost(actual, pred, cap):
    """
    1. Choose (bikes, docks) that minimises α·missPU + β·missDO on the prediction
    2. Evaluate that same allocation on the actual demand
       · actual  = true [PU_ct, DO_ct]
       · pred    = predicted [PU_ct, DO_ct]
       · cap     = station capacity (bikes + docks)
    """
    best_cost = np.inf
    for bikes in range(cap + 1):
        docks = cap - bikes
        cost_pred = ALPHA * max(pred[0] - bikes, 0) + BETA * max(pred[1] - docks, 0)
        if cost_pred < best_cost:
            best_cost = cost_pred
            best_pair = (bikes, docks)

    b, d = best_pair
    return ALPHA * max(actual[0] - b, 0) + BETA * max(actual[1] - d, 0)

In [133]:
# Define the model zoo and broad hyper-parameter search spaces
from scipy.stats import randint, uniform, loguniform

# Base models
base_models = {
    "Linear"    : LinearRegression(),
    "Ridge"     : Ridge(),
    "Lasso"     : ElasticNet(l1_ratio=1.0, max_iter=5000),
    "ElasticNet": ElasticNet(max_iter=5000),
    "KNN"       : KNeighborsRegressor(),
    "Tree"      : DecisionTreeRegressor(random_state=200),
    "RF"        : RandomForestRegressor(random_state=200, n_jobs=-1),
    "GB"        : GradientBoostingRegressor(random_state=200),
    "NN"        : MLPRegressor(max_iter=4000, random_state=200,
                               early_stopping=True, n_iter_no_change=20)
}

# Wrap each base model with MultiOutputRegressor to handle multi-target regression
models = {k: MultiOutputRegressor(v, n_jobs=-1) for k, v in base_models.items()}

# Hidden layer sizes for NN (1 or 2 layers, avoiding illegal combinations)
hl_single = [(h,) for h in range(40, 181, 20)]
hl_double = [(h1, h2) for h1 in range(40, 181, 20) for h2 in range(20, 161, 20)]

# Hyperparameter distributions for RandomizedSearchCV
param_dist = {
    "Linear":     {},  # Nothing to tune
    "Ridge":      {"est__estimator__alpha": loguniform(1e-3, 1e2)},
    "Lasso":      {"est__estimator__alpha": loguniform(1e-4, 1),
                   "est__estimator__l1_ratio": uniform(0, 1)},
    "ElasticNet": {"est__estimator__alpha": loguniform(1e-4, 1),
                   "est__estimator__l1_ratio": uniform(0, 1)},
    "KNN":        {"est__estimator__n_neighbors": randint(1, 21),
                   "est__estimator__weights": ["uniform", "distance"],
                   "est__estimator__p": [1, 2]},
    "Tree":       {"est__estimator__max_depth": randint(2, 15),
                   "est__estimator__min_samples_leaf": randint(1, 10)},
    "RF":         {"est__estimator__n_estimators": randint(200, 601),
                   "est__estimator__max_depth": randint(3, 15),
                   "est__estimator__max_features": ["sqrt", "log2", None]},
    "GB":         {"est__estimator__n_estimators": randint(200, 601),
                   "est__estimator__learning_rate": loguniform(0.01, 0.3),
                   "est__estimator__max_depth": randint(2, 6),
                   "est__estimator__subsample": uniform(0.5, 0.5)},
    "NN":         {"est__estimator__alpha": loguniform(1e-6, 1e-1),
                   "est__estimator__hidden_layer_sizes": hl_single + hl_double}
}

In [134]:
# Run a MUCH deeper search: 100 draws over 5-fold CV (takes longer)
outer_cv   = KFold(n_splits=5, shuffle=True, random_state=200)
records    = []

for name, estimator in models.items():
    pipe = Pipeline([("scale", StandardScaler()), ("est", estimator)])

    # pick sensible n_iter: 100 for sizeable spaces, 1 when there is nothing to tune
    n_iter = 100 if param_dist[name] else 1

    search = RandomizedSearchCV(
        pipe, param_distributions=param_dist[name],
        n_iter=n_iter, cv=outer_cv,
        scoring="neg_root_mean_squared_error",
        random_state=123, n_jobs=-1, verbose=0
    )
    search.fit(X_train, y_train)

    y_pred = search.best_estimator_.predict(X_test)
    records.append({
        "Model"      : name,
        "BestParams" : search.best_params_,
        "MSE_PU_ct"  : mean_squared_error(y_test["PU_ct"], y_pred[:, 0]),
        "MSE_DO_ct"  : mean_squared_error(y_test["DO_ct"], y_pred[:, 1]),
        "y_pred"     : y_pred
    })

df_pred = pd.DataFrame(records)[["Model", "BestParams", "MSE_PU_ct", "MSE_DO_ct"]]
df_pred

,Model,BestParams,MSE_PU_ct,MSE_DO_ct
0,Linear,{},55.580504,69.546556
1,Ridge,{'est__estimator__alpha': 2.8741471117024773},55.721675,69.474573
2,Lasso,"{'est__estimator__alpha': 0.4051776587763117, ...",56.940808,69.849760
3,ElasticNet,"{'est__estimator__alpha': 0.4051776587763117, ...",56.940808,69.849760
4,KNN,"{'est__estimator__n_neighbors': 14, 'est__esti...",61.515448,75.110119
5,Tree,"{'est__estimator__max_depth': 2, 'est__estimat...",81.508918,98.787005
6,RF,"{'est__estimator__max_depth': 3, 'est__estimat...",69.231721,84.703795
7,GB,{'est__estimator__learning_rate': 0.0144956388...,75.658569,97.017923
8,NN,{'est__estimator__alpha': 9.79328536637393e-05...,77.453420,92.303277


In [135]:
# Business cost with capacity K = 20 (α = 2, β = 3)
for r in records:
    r["Cost_Capacity_20"] = np.mean([
        decision_cost(a, p, cap=20) for a, p in zip(y_test.values, r["y_pred"])
    ])

df_cost20 = pd.DataFrame(records)[["Model", "Cost_Capacity_20"]]
df_cost20

,Model,Cost_Capacity_20
0,Linear,70.138889
1,Ridge,70.138889
2,Lasso,70.138889
3,ElasticNet,70.138889
4,KNN,70.138889
5,Tree,69.916667
6,RF,70.027778
7,GB,69.833333
8,NN,70.166667


In [136]:
# Business cost with capacity K = 30 (α = 2, β = 3)
for r in records:
    r["Cost_Capacity_30"] = np.mean([
        decision_cost(a, p, cap=30) for a, p in zip(y_test.values, r["y_pred"])
    ])

df_cost30 = pd.DataFrame(records)[["Model", "Cost_Capacity_30"]]
df_cost30

,Model,Cost_Capacity_30
0,Linear,50.138889
1,Ridge,50.222222
2,Lasso,50.222222
3,ElasticNet,50.222222
4,KNN,50.694444
5,Tree,50.750000
6,RF,49.972222
7,GB,49.583333
8,NN,51.138889


In [137]:
# Final performance table for the report
summary = (
    df_pred
      .merge(df_cost20, on="Model")
      .merge(df_cost30, on="Model")
      .sort_values("Cost_Capacity_20")      # sort by main business metric
      .reset_index(drop=True)
)

summary

,Model,BestParams,MSE_PU_ct,MSE_DO_ct,Cost_Capacity_20,Cost_Capacity_30
0,GB,{'est__estimator__learning_rate': 0.0144956388...,75.658569,97.017923,69.833333,49.583333
1,Tree,"{'est__estimator__max_depth': 2, 'est__estimat...",81.508918,98.787005,69.916667,50.750000
2,RF,"{'est__estimator__max_depth': 3, 'est__estimat...",69.231721,84.703795,70.027778,49.972222
3,Linear,{},55.580504,69.546556,70.138889,50.138889
4,Ridge,{'est__estimator__alpha': 2.8741471117024773},55.721675,69.474573,70.138889,50.222222
5,Lasso,"{'est__estimator__alpha': 0.4051776587763117, ...",56.940808,69.849760,70.138889,50.222222
6,ElasticNet,"{'est__estimator__alpha': 0.4051776587763117, ...",56.940808,69.849760,70.138889,50.222222
7,KNN,"{'est__estimator__n_neighbors': 14, 'est__esti...",61.515448,75.110119,70.138889,50.694444
8,NN,{'est__estimator__alpha': 9.79328536637393e-05...,77.453420,92.303277,70.166667,51.138889


The evaluation of model performance in this context involves balancing prediction accuracy (measured by Mean Squared Error, MSE) and decision cost (determined by the optimal allocation of bikes and docks at the station). While a model with better prediction accuracy generally seems preferable, it does not always lead to a lower decision cost. This is because the decision cost function also accounts for the allocation strategy and the penalties (α and β) for missed pickups and drop-offs. As a result, a model with lower MSE may still incur higher costs if it fails to accurately balance pickups and drop-offs.

Interestingly, the differences in decision cost among the models are relatively small, whereas the differences in MSE are quite significant. This indicates that both prediction accuracy and decision cost should be considered when selecting the best model. Ultimately, the choice may depend on what the sponsors or clients value more—predictive accuracy or cost efficiency. By weighing these priorities, we can make a more informed and tailored decision.